# Botometer checker
This notebook allows you to run up to 2,000 accounts through [Botometer](https://botometer.iuni.iu.edu/#!/) per day, and populates a Google Sheet with the results in real time using [pygsheets](https://pygsheets.readthedocs.io/en/stable/spreadsheet.html).

In [1]:
#imports
import tweepy
import json 
import sys
import numpy as np
import pandas as pd
try:
    import botometer
except:
    !{sys.executable} -m pip install botometer
    import botometer
    
try:
    import pygsheets
except:
    !{sys.executable} -m pip install pygsheets
    import pygsheets

#log into google sheets
gc = pygsheets.authorize(service_file='pygsheets_authfile.json')

In [2]:
#log in to twitter
key_file = 'keys.json'
with open(key_file) as f:
    keys = json.load(f)

try:
    auth = tweepy.OAuthHandler(keys["consumer_key"], keys["consumer_secret"])
    auth.set_access_token(keys["access_token"], keys["access_token_secret"])
    api = tweepy.API(auth)
    print("Your username is:", api.auth.get_username())
except TweepError as e:
    logging.warning("There was a Tweepy error. Double check your API keys and try again.")
    logging.warning(e)

Your username is: s_onnet


In [3]:
#instantiate botometer

mashape_key = "6943c3ed5amsh037a969e4eb3116p141bf5jsne4707bf51988"

twitter_app_auth = {
    'consumer_key': keys["consumer_key"],
    'consumer_secret': keys["consumer_secret"],
    'access_token': keys["access_token"],
    'access_token_secret': keys["access_token_secret"],
  }
bom = botometer.Botometer(wait_on_ratelimit=True,
                          mashape_key=mashape_key,
                          **twitter_app_auth)

In [4]:
#testing on one user
bom.check_account('@anjbanerjee')

{'cap': {'english': 0.8469348665516686, 'universal': 0.7739048284466749},
 'categories': {'content': 0.8908083596949135,
  'friend': 0.860476553892247,
  'network': 0.31969317474163084,
  'sentiment': 0.8630244302008979,
  'temporal': 0.8999315382318549,
  'user': 0.9566079256069624},
 'display_scores': {'content': 4.5,
  'english': 4.6,
  'friend': 4.3,
  'network': 1.6,
  'sentiment': 4.3,
  'temporal': 4.5,
  'universal': 4.6,
  'user': 4.8},
 'scores': {'english': 0.9298669453061029, 'universal': 0.9144161859602663},
 'user': {'id_str': '1018896250257723392', 'screen_name': 'AnjBanerjee'}}

In [5]:
#read in csv of followers
users = pd.read_csv('pulwama_followers.csv')
users.drop(columns=['Unnamed: 6', 'Unnamed: 7'])
#get array of first columnn
fol1 = list(users.iloc[:,0].dropna().values)
fol2 = list(users.iloc[:,1].dropna().values)
fol3 = list(users.iloc[:,2].dropna().values)
fol4 = list(users.iloc[:,3].dropna().values)
fol5 = list(users.iloc[:,4].dropna().values)
fol6 = list(users.iloc[:,5].dropna().values)

all_followers = list(set(fol3 + fol4 + fol5 + fol6)) #minus fol1 + fol2
all_followers.sort()
len(all_followers)

9474

In [25]:
wks1 = gc.open('Botometer scores')[1]
wks2 = gc.open('Botometer scores')[5]
wks3 = gc.open('Botometer scores')[6]
#sh = worksheet('Botometer scores')
checked_users = wks1.get_col(col=1, include_tailing_empty=False)
checked_users = checked_users + wks2.get_col(col=1, include_tailing_empty=False)
checked_users = checked_users + wks3.get_col(col=1, include_tailing_empty=False)

print(len(checked_users))
checked_users.sort()
all_followers.sort()


2633


In [26]:
print(checked_users.index('@FahadAkhterr'))
all_followers.index('@FahadAkhterr')

1757


389

In [27]:
need_to_check = all_followers
for i in need_to_check:
    if i in checked_users:
        need_to_check.remove(i)
len(need_to_check)

need_to_check.sort()

In [28]:
print(len(need_to_check))
print(need_to_check[0])
need_to_check.index('@FahadAkhterr')

7059
@0shesheB


194

In [258]:
def run_botometer(users, idx, sheet):
    """Runs array of users through botometer, 
    filling results in Google sheet as it goes """
    
    col_names = ['id','english_score', 'universal_score','friend_score',
                 'temporal_score','user_score','network_score', 'content_score']
    scores_df = pd.DataFrame(columns=col_names, index=users)
    
    #add column labels to Google Sheet if idx = 0
    if idx==0:
        sheet.insert_rows(row=idx, number=1, values=['user'] + col_names)
        idx+=1
    for user in users:
        try: 
            score = bom.check_account(user)
            scores_df.loc[user] = [score['user']['id_str'], score['scores']['english'], 
                            score['scores']['universal'], score['display_scores']['friend'],
                            score['display_scores']['temporal'], score['display_scores']['user'],
                            score['display_scores']['network'], score['display_scores']['content']]
            sheet.insert_rows(row=idx, number=1, values=[user] + list(scores_df.loc[user]))
        except:
            scores_df.loc[user]['id'] = 'User disabled'
            sheet.insert_rows(row=idx, number=1, values=[user, 'User disabled'])
        idx+=1
        
    return scores_df


In [260]:
#CAREFUL WITH THIS CELL! BOTOMETER LIMITS YOU TO 2,000 HANDLES PER DAY
sheet = gc.open("Botometer scores")
worksheet = sheet.add_worksheet['Next 1,9000 (Wed 4/17)'] #Change title 
#scores_df = run_botometer(need_to_check[0:1900], 0, worksheet) #uncomment here to RUN
scores_df.head()

ServerNotFoundError: Unable to find the server at sheets.googleapis.com

In [269]:
scores_df

,id,english_score,universal_score,friend_score,temporal_score,user_score,network_score,content_score
@___Q3R,User disabled,NaN,NaN,NaN,NaN,NaN,NaN,NaN
@__abo3ziz,2801587087,0.569732,0.799058,2.7,0.6,2.7,3.2,1.2


In [243]:
#Test worksheet
sheet = gc.open("Botometer scores")
worksheet = sheet[0]
worksheet.insert_rows(row=1, number=1, values=list(scores_df.loc['@__abo3ziz']))

In [219]:
through_4_df = df
through_4_df

,user,english_score,universal_score
0,@___Q3R,User disabled,User disabled
1,@__abo3ziz,0.421517,0.421517
2,@__dk0,0.70642,0.70642
3,@__uo9,0.610976,0.610976
